## **Turi Create Image Classifier on Skafos**

This notebook does the following:

1. Loads 25,000 cat and dog images.
2. Builds a classification model to predict whether an image is a cat or a dog.
3. Converts the model to CoreML and saves it to the Skafos framework.

The example below is based on [Turi Create's Image Classifier](https://apple.github.io/turicreate/docs/userguide/image_classifier/).

In [ ]:
from utilities.load_turicreate import *
tc = install_turicreate(timeout = 500, retries = 2)
import urllib
import tarfile
import coremltools
import utilities.save_models as sm
from skafossdk import *

In [ ]:
ska  = Skafos() # initialize Skafos

### 1. **Load the data**
The data for this example are 25000 images, 12500 Cats and 12500 Dogs. The original data set is [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765) and we have also included included it in the public bucket listed below. To retrain this model using your own data, you will want to replace the `data_url` and `data_path` below with the appropriate code to source your own data. An example of how to do this can be found in `userguide/images_in_turicreate.ipynb`

In [ ]:
# Fetch training data from an AWS bucket publicly hosted by Metis Machine.

data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/PetImages.tar.gz"
data_path = "PetImages.tar.gz"

retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
# Load images (Note: you can ignore 'Not a JPEG file' errors)
data = tc.image_analysis.load_images('PetImages', with_path=True, ignore_failure = True)

# From the path-name, create a label column. This labels each image as either a dog or a cat. 
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else 'cat')

# test train split
train_data, test_data = data.random_split(0.8)

### 2. **Build the model**
We pass the data to the `tc.image_classifer.create` function and specify the labels needed to properly run the model. More about this function and the parameters it takes can be found [here](https://apple.github.io/turicreate/docs/userguide/image_classifier/).


_Note: The code below has been commented because we highly recommend running this model using a GPU.  If you uncomment this line and run this notebook as written, it will take approximately 90 minutes, and will re-create the cat/dog predictor model. Once you have We encourage you to do this once you've changed the data to reflect your use case._

In [ ]:
# Train an image classification model, specifying the name of the 'target'/'label' column
#model = tc.image_classifier.create(train_data, target='label')

### 3. **Save the model**
Once your model has been created, it must be saved to the Skafos framework via the code below. 

In [ ]:
## Save the model for later use in Skafos
#coreml_model_name = 'image_classifier.mlmodel'
#res = model.export_coreml(coreml_model_name)

## use coremltools utility function to convert model weights to half-precision
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# compress the model
#compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# save to Skafos
#sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
#                                compressed_model = compressed_model,
#                                permissions = 'public')